In [1]:
from pyspark.sql import SparkSession

OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k


In [2]:
spark = SparkSession.builder \
            .appName("ProcessDataWarehouse") \
            .getOrCreate()

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-fb1de3e6-1fd4-498f-8736-84f764cb99d9;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.2 in central
	found org.apache.kafka#kafka-clients;2.6.0 in central
	found com.github.lub

In [3]:
psql_host = "postgres"
psql_port = "5432"
psql_database = "pyne"
psql_jdbc_url = f"jdbc:postgresql://{psql_host}:{psql_port}/{psql_database}"
properties = {"user": "pyne","password": "pyne","driver": "org.postgresql.Driver"}
topic = "finance.broker.transactions.customers"

In [4]:
customer_stream_df = spark.read \
                          .format("delta") \
                          .option("path", f'/home/jovyan/work/datalake/silver/{topic}') \
                          .load()

In [5]:
customer_stream_df.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- customer_username: string (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- customer_gender: string (nullable = true)
 |-- customer_address: string (nullable = true)
 |-- customer_purchase_price: float (nullable = true)
 |-- customer_country: string (nullable = true)
 |-- year: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day: string (nullable = true)



In [6]:
customer_stream_df.write.jdbc(url=psql_jdbc_url, table="pyne.customer", mode="append", properties=properties)